## Hop-pub test

_24th August 2021_ <br>
Sebastian Lara-Torres<br>
Melih Kara

**To do:** 
- Implement Heartbeat messages on a seperate continuously running script? <br>
- Should Alert messages be published this way?
- Accept Datetime object and convert into str
- Accept other input types?
- We do not have different topics for different tiers, the content should be modified depending on the tier right? It might make sense to refactor according to this.

In [1]:
import hop_pub_v02 as hop_pub

In [2]:
# by default, it still publishes something
# can be randomized
publisher = hop_pub.Publish_Observation(welcome=True)


### Publish SNEWS Observation Messages ###
 Your Python version:3.8.5 (default, Jan 27 2021, 15:41:15) 
[GCC 9.3.0]
Current hop-client version:0.4.0
             snews version:0.0.1

 Publishing to kafka.scimma.org
 Observation Topic: kafka://kafka.scimma.org/snews.alert-test

 Submitting messages to the following Tiers;
Significance_Tier & Coincidence_Tier & Timing_Tier



Publishing option can be changed.

In [3]:
publisher.publish_to['Timing_Tier'] = False
print(publisher.publish_to)

{'Significance_Tier': True, 'Coincidence_Tier': True, 'Timing_Tier': False}


In [4]:
# default dictionary
publisher.msg_dict

{'message_id': 0,
 'detector_id': 0,
 'sent_time': '01/01/01 01:01:01',
 'neutrino_time': '01/01/01 01:01:01',
 'machine_time': '01/01/01 01:01:01',
 'location': 'none',
 'p_value': 0,
 'status': 'none',
 'content': 'none',
 'time-series': 0}

### Message as a dictionary

In [5]:
message = {'machine_time':'24/08/2021 15:49:55',
           'status': 'ON'} # To Do: accept datetime ovject and convert into str
publisher = hop_pub.Publish_Observation(msg=message)
# Only the value that is changed is overwritten
publisher.msg_dict

{'message_id': 0,
 'detector_id': 0,
 'sent_time': '01/01/01 01:01:01',
 'neutrino_time': '01/01/01 01:01:01',
 'machine_time': '24/08/2021 15:49:55',
 'location': 'none',
 'p_value': 0,
 'status': 'ON',
 'content': 'none',
 'time-series': 0}

In [6]:
publisher.publish_to_tiers()


Publishing OBS message to Significance_Tier:
detector_id         :0
machine_time        :24/08/2021 15:49:55
neutrino_time       :01/01/01 01:01:01
status              :ON
p_value             :0


Exception: 1 messages remain unsent, some data may have been lost!

## Issue
The `Stream` fails when trying to publish several messages in a short time interval.

In [8]:
from hop import Stream
observation_topic = 'kafka://kafka.scimma.org/snews.experiments-test'

In [13]:
message1 = {'test_key1':'test_data1', 'test_key2':'test_data2'}
message2 = {'test_key1':'test_data1', 'test_key2':'test_data2', 'test_key3':'test_data3'}
message3 = {'test_key2':'test_data2'}

In [19]:
# publishing one message works
stream = Stream(persist=False)
with stream.open(observation_topic, "w") as s:
    s.write(message1)

In [20]:
# one needs to wait couple of seconds before submitting another message
# following also fails in both persist=False or True cases
stream = Stream(persist=False)
with stream.open(observation_topic, "w") as s:
    s.write(message1)
    s.write(message2)
    s.write(message3)    

Exception: 3 messages remain unsent, some data may have been lost!

In [21]:
# open a stream that does not persist
# stream = Stream(persist=False)
for i, data in enumerate([message1, message2, message3]):
    stream = Stream(persist=False)
    with stream.open(observation_topic, "w") as s:
        s.write(data)
    print(f'\nThe {i+1}th message is sent!')
    for k,v in data.items():
        print(f'{k:<20s}:{v}')


The 1th message is sent!
test_key1           :test_data1
test_key2           :test_data2

The 2th message is sent!
test_key1           :test_data1
test_key2           :test_data2
test_key3           :test_data3

The 3th message is sent!
test_key2           :test_data2


In [65]:
def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [66]:
isnotebook()

True

In [16]:
from IPython.display import HTML, display

giphy_experiments = "https://im.ezgif.com/tmp/ezgif-1-5da44911f4c4.gif"
giphy_snews = "https://im.ezgif.com/tmp/ezgif-1-719febc3dfb2.gif"
answer = input('Are you authorized to send alert messages? [y/n]')
if answer.lower() == 'y':
    display(HTML(f'<img src={giphy_snews}>'))
else:
    display(HTML(f'<img src={giphy_experiments}>'))

Are you authorized to send alert messages? [y/n]y


In [12]:
from html import escape # Python 3 only :-)

class Todo:
    def __init__(self):
        self.items = []

    def add(self, text, completed):
        self.items.append({'text': text, 'completed': completed})

    def _repr_html_(self):
        return "<ol>{}</ol>".format("".join("<li>{} {}</li>".format(
            "☑" if item['completed'] else "☐",
            escape(item['text'])
        ) for item in self.items))

my_todo = Todo()
my_todo.add("Buy milk", False)
my_todo.add("Do homework", False)
my_todo.add("Play video games", True)

my_todo

---

In [59]:
from dotenv import load_dotenv
from datetime import datetime
from collections import namedtuple
import os

class TimeStuff:
    def __init__(self, env=None):
        set_env(env)
        self.snews_t_format = os.getenv("TIME_STRING_FORMAT")
        self.hour_fmt = "%H:%M:%S"
        self.date_fmt = "%y_%m_%d"
        
        self.get_datetime = datetime.utcnow()
        self.get_snews_time = lambda fmt=self.snews_t_format : datetime.utcnow().strftime(fmt)
        self.get_hour = lambda fmt=self.hour_fmt : datetime.utcnow().strftime(fmt)
        self.get_date = lambda fmt=self.date_fmt : datetime.utcnow().strftime(fmt)

In [60]:
datetime.utcnow().strftime("%H:%M:%S")

'09:07:07'

In [62]:
a = TimeStuff()
print(a.get_datetime)
print(a.get_snews_time())
print(a.get_hour())
print(a.get_date())

2021-08-27 09:07:15.398744
21/08/27 09:07:15
09:07:15
21_08_27


In [33]:
b = datetime.utcnow() #a.get_snews_time_str()
for i in dir(b):
    if '__' not in i:
        print(f"{i}\t", getattr(b, i))

astimezone	 <built-in method astimezone of datetime.datetime object at 0x7f8b13757de0>
combine	 <built-in method combine of type object at 0x92ab80>
ctime	 <built-in method ctime of datetime.datetime object at 0x7f8b13757de0>
date	 <built-in method date of datetime.datetime object at 0x7f8b13757de0>
day	 27
dst	 <built-in method dst of datetime.datetime object at 0x7f8b13757de0>
fold	 0
fromisocalendar	 <built-in method fromisocalendar of type object at 0x92ab80>
fromisoformat	 <built-in method fromisoformat of type object at 0x92ab80>
fromordinal	 <built-in method fromordinal of type object at 0x92ab80>
fromtimestamp	 <built-in method fromtimestamp of type object at 0x92ab80>
hour	 8
isocalendar	 <built-in method isocalendar of datetime.datetime object at 0x7f8b13757de0>
isoformat	 <built-in method isoformat of datetime.datetime object at 0x7f8b13757de0>
isoweekday	 <built-in method isoweekday of datetime.datetime object at 0x7f8b13757de0>
max	 9999-12-31 23:59:59.999999
microsecond	 

In [37]:
b.hour()

8